# Train Logistic Regression classifier with Apache SparkML


In [5]:
import pip


def install(package):
    if hasattr(pip, 'main'):
        pip.main(['install', package])
    else:
        pip._internal.main(['install', package])


install('pyspark==3.1.2')
install('wget==3.2')
install('pyspark2pmml==0.5.1')

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=a7f0b4ff948470cc010a81cbad8c6424a17cecd32f233486059d500d39029167
  Stored in directory: /home/jovyan/.cache/pip/wheels/df/88/9e/58ef1f74892fef590330ca0830b5b6d995ba29b44f977b3926
Successfully built pyspark
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.1.1
    Uninstalling pyspark-3.1.1:
      Successfully uninstalled pyspark-3.1.1


You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


  Created wheel for pyspark2pmml: filename=pyspark2pmml-0.5.1-py3-none-any.whl size=2418 sha256=b0c29f9710f3e0809f823b3271b3d9bbab8ab0ce4a5655393a3842dfc8c5ec50
  Stored in directory: /home/jovyan/.cache/pip/wheels/fd/9f/02/ce0d5f8237f50df7c58196d4e88f7fb59f6d077372de102fb1
Successfully built pyspark2pmml


You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


In [7]:
from pyspark import SparkContext, SparkConf, SQLContext
import os
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark2pmml import PMMLBuilder
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import MinMaxScaler
import wget
import sys
import shutil
import site

In [13]:
if sys.version[0:3] == '3.8':
    url = 'https://github.com/jpmml/jpmml-sparkml/releases/download/1.7.2/'
    + 'jpmml-sparkml-executable-1.7.2.jar'
    wget.download(url)
    shutil.copy('jpmml-sparkml-executable-1.7.2.jar', site.getsitepackages()[0]
                + '/pyspark/jars/')
elif sys.version[0:3] == '3.6':
    url = 'https://github.com/jpmml/jpmml-sparkml/releases/download/1.5.12/'
    + 'jpmml-sparkml-executable-1.5.12.jar'
    wget.download(url)
else:
    raise Exception('Currently only python 3.6 and 3.8 is supported, in case '
                    + 'you need a different version please open an issue at '
                    + 'https://github.com/elyra-ai/component-library/issues')

Exception: Currently only python 3.6 and 3.8 is supported, in case you need a different version please open an issue at https://github.com/elyra-ai/component-library/issues

In [ ]:
data_parquet = os.environ.get('data_parquet',
                              'data.parquet')  # input file name (parquet)
master = os.environ.get('master',
                        "local[*]")  # URL to Spark master
model_target = os.environ.get('model_target',
                              "model.xml")  # model output file name
data_dir = os.environ.get('data_dir',
                          '../../data/')  # temporary directory for data
input_columns = os.environ.get('input_columns',
                               '["x", "y", "z"]')  # input columns to consider

In [ ]:
conf = SparkConf().setMaster(master)
if sys.version[0:3] == '3.6':
    conf.set("spark.jars", 'jpmml-sparkml-executable-1.5.12.jar')

sc = SparkContext.getOrCreate(conf)
sqlContext = SQLContext(sc)
spark = sqlContext.sparkSession

In [ ]:
df = spark.read.parquet(data_dir + data_parquet)

In [ ]:
# register a corresponding query table
df.createOrReplaceTempView('df')

In [ ]:
from pyspark.sql.types import DoubleType
df = df.withColumn("x", df.x.cast(DoubleType()))
df = df.withColumn("y", df.y.cast(DoubleType()))
df = df.withColumn("z", df.z.cast(DoubleType()))

In [ ]:
splits = df.randomSplit([0.8, 0.2])
df_train = splits[0]
df_test = splits[1]

In [ ]:
indexer = StringIndexer(inputCol="class", outputCol="label")

vectorAssembler = VectorAssembler(inputCols=eval(input_columns),
                                  outputCol="features")

normalizer = MinMaxScaler(inputCol="features", outputCol="features_norm")

In [ ]:
# import org.apache.spark.ml.feature.RFormula

# val irisData = spark.read.format("csv").option("header", "true").
# option("inferSchema", "true").load("Iris.csv")
# val irisSchema = irisData.schema

# val rFormula = new RFormula().setFormula("Species ~ .")
# val dtClassifier = new DecisionTreeClassifier().setLabelCol(
# rFormula.getLabelCol).setFeaturesCol(rFormula.getFeaturesCol)
# val pipeline = new Pipeline().setStages(Array(rFormula, dtClassifier))

In [ ]:
lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

In [ ]:
pipeline = Pipeline(stages=[indexer, vectorAssembler, normalizer, lr])

In [ ]:
model = pipeline.fit(df_train)

In [ ]:
prediction = model.transform(df_train)

In [ ]:
binEval = MulticlassClassificationEvaluator(). \
    setMetricName("accuracy"). \
    setPredictionCol("prediction"). \
    setLabelCol("label")

binEval.evaluate(prediction)

In [ ]:
pmmlBuilder = PMMLBuilder(sc, df_train, model)
pmmlBuilder.buildFile(data_dir + model_target)